In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:

folder_path = "MPIIFaceGaze_preprocessed/Image/p00/face"


images = [img for img in os.listdir(folder_path) if img.endswith(".jpg")]
image_data = []

for img_name in images:
    img_path = os.path.join(folder_path, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img,(224,224))
    image_data.append(img)

image_data = np.array(image_data)
print("Stored the data into an variable is successfully done")
# Split the dataset into train and test sets
train_images, test_images = train_test_split(image_data, test_size=0.2, random_state=42)

# Normalize pixel values between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0
print("The images are sent for training and Testing")



Stored the data into an variable is successfully done
The images are sent for training and Testing


In [3]:
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.nn.functional as F

class FaceDataset(Dataset):
    def __init__(self,images):
        self.images = images
        self.labels = torch.randn(len(images),2)
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img = self.images[idx]
        if isinstance(img, np.ndarray):
            img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)  # From HWC to CHW
        label = self.labels[idx]
        return img, label

        

train_dataset = FaceDataset(train_images)
test_dataset = FaceDataset(test_images)
print("Training Successfully done")
train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=16)
print("Training Loader Successfully Done")

sample_batch, _ = next(iter(train_loader))
print("Sample batch shape:", sample_batch.shape)


Training Successfully done
Training Loader Successfully Done
Sample batch shape: torch.Size([16, 3, 224, 224])


In [4]:
class GazeCNN(nn.Module):
    def __init__(self):
        super(GazeCNN,self).__init__()
        self.conv1 = nn.Conv2d(3,32, kernel_size=3,padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,kernel_size = 3,padding=1)
        self.fc1 = nn.Linear(64*56*56,128)
        self.fc2 = nn.Linear(128,2)
    def forward(self, x):
        x = F.relu(self.conv1(x))     # Conv1 + ReLU
        x = self.pool(x)              # Max Pooling
        x = F.relu(self.conv2(x))     # Conv2 + ReLU
        x = self.pool(x)              # Max Pooling again
        x = x.view(x.size(0), -1)     # Flatten
        x = F.relu(self.fc1(x))       # FC Layer
        x = self.fc2(x)               # Final output
    print("GazeCNN Module Training Successfully done")
        return x

        

In [ ]:
modal = GazeCNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(modal.parameters(),lr=0.001)
for epoch in range(10):
    modal.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        outputs = modal(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module Training continues
GazeCNN Module

In [ ]:
import cv2
import torch
from torchvision import transforms

model = GazeCNN()
model.load_state_dict(torch.load("gaze_model.pth"))
model.eval()

cap = cv2.VideoCapture(0)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img = cv2.resize(frame, (224, 224))
    img_tensor = transform(img).unsqueeze(0)  # Add batch dim

    with torch.no_grad():
        output = model(img_tensor)
        prediction = torch.argmax(output, dim=1).item()
        print("Predicted class:", prediction)

    cv2.imshow("Webcam", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print("OpenCV working")